In [0]:
import json
from delta.tables import DeltaTable
from pyspark.sql.functions import sha2, concat_ws, current_timestamp, lit
from pyspark.sql.functions import struct, to_json, sha2, col, current_timestamp
from functions import create_table_if_not_exists

history_df = DeltaTable.forName(spark, "edsm.silver.powerPlay").history()
versions = [r.version for r in history_df.select("version").collect()]

for v in sorted(versions):
    df = (
        spark.read
        .format("delta")
        .option("versionAsOf", v)
        .table("edsm.silver.powerPlay")
    )
    dupes = (
        df.groupBy("id", "id64", "name", "power", "powerState", "state")
        .count()
        .filter("count > 1")
    )
    if dupes.count() > 0:
        print(f"Version {v} has duplicates")
        dupes.show(truncate=False)
